In [ ]:
from openbb_terminal.sdk import openbb
import plotly.graph_objects as go
import yfinance as yf
import pandas as pd

import plotly.express as px
import numpy as np

import datetime


In [ ]:
# 3D plotly examples https://plotly.com/python/3d-scatter-plots/
# Generate some sample data for 3D plot
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
x, y = np.meshgrid(x, y)
z = np.sin(np.sqrt(x**2 + y**2))

# Create a 3D surface plot
fig = go.Figure(data=[go.Surface(z=z)])

# Update layout to set axis labels
fig.update_layout(
    title='3D Surface Plot',
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z'),
    )
)

# Show the plot
fig.show()

In [ ]:
import plotly.graph_objects as go 
# Read data from a csv
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')

fig = go.Figure(data=[go.Surface(z=z_data.values)])
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()

In [ ]:
# risk return profile and examples

import numpy as np
import matplotlib.pyplot as plt

# Define the risk (volatility) and return (expected yield) ragces
risk_range = np.linspace(0, 0.3, 20)  # Example: Volatility from 0% to 30%
return_range = np.linspace(0, 0.2, 20)  # Example: Expected yield from 0% to 20%

# Create a grid of points in the risk-return space
risk, return_ = np.meshgrid(risk_range, return_range)

# Calculate a vector field based on a simple model (e.g., linear relationship)
risk_vector = risk
return_vector = 0.5 * return_ + 0.2 * risk

# Create a quiver plot to visualize the vector field
plt.figure(figsize=(10, 6))
plt.quiver(risk, return_, risk_vector, return_vector, scale=30, color='r', alpha=0.9)
plt.xlabel('Risk (Volatility)')
plt.ylabel('Return (Expected Yield)')
plt.title('Portfolio Risk-Return Profile')

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import datetime
current_timestamp = datetime.datetime.now()


formatted_date = current_timestamp.strftime("%Y-%m-%d")
print("Current timestamp:", formatted_date)

# Sample historical daily price data for a commodity futures contract
# Replace this with your actual price data using openbb
prices = openbb.futures.historical(symbols='GC', start_date='2022-01-01', end_date=formatted_date)
prices['price_changes'] = prices['Close'].pct_change().fillna(0)  # Fill NaN with 0 for the first day

# Define a time vector (days)
time = np.arange(len(prices['price_changes']))

# Create a line plot of daily price changes over time
plt.figure(figsize=(10, 6))
plt.plot(time, prices['price_changes'], color='b', linestyle='-', marker='o')
plt.xlabel('Time (Days)')
plt.ylabel('Daily Price Change')
plt.title('Commodity Futures Daily Price Changes Over Time')

plt.show()


In [ ]:
import plotly.express as px
import pandas as pd

# Sample historical daily price data for a commodity futures contract
# Replace this with your actual price data using openbb
prices = openbb.futures.historical(symbols='GC', start_date='2020-01-01', end_date=formatted_date)
prices['price_changes'] = prices['Close'].pct_change().fillna(0)  # Fill NaN with 0 for the first day

# Create a DataFrame with the data
df = pd.DataFrame({'Date': prices.index, 'Price Change': prices['price_changes']})

# Create a line plot using Plotly
fig = px.line(df, x='Date', y='Price Change', title='Commodity Futures Daily Price Changes Over Time')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Daily Price Change')

# Show the Plotly plot
fig.show()


In [ ]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import brentq
from scipy.stats import beta
import matplotlib.pyplot as plt

# Constants
α, a, c = 0.8, 1.0, 2.0
mc_draw_size = 250
gridsize = 150
grid_max = 35
tol = 1e-4

# Grid for x values
grid = np.linspace(a, grid_max, gridsize)

# Beta distribution for shock observations
beta_dist = beta(5, 5)
Z = a + beta_dist.rvs(mc_draw_size) * c

# Define inverse demand function P(x) and initialize prices
P = lambda x: 1.0 / x
prices = np.array([P(x) for x in grid])

# The T operator
def T(p_array):
    new_prices = np.empty_like(p_array)

    # Create interpolation function
    p_interp = interp1d(grid, p_array, bounds_error=False, fill_value="extrapolate")

    # Loop over each x in grid to update prices
    for i, x in enumerate(grid):
        def h(q):
            # Use the interpolated function and not the array
            return q - max(α * np.mean(p_interp(α * (x - 1/q) + Z)), 1/x)
        
        # Find the root of h and update the prices
        new_prices[i] = brentq(h, 1e-8, 100)
    
    return new_prices

# Iteration to find the equilibrium prices
error = tol + 1
while error > tol:
    new_prices = T(prices)
    error = np.max(np.abs(new_prices - prices))
    prices = new_prices

# Plotting the results
fig, ax = plt.subplots()

ax.plot(grid, 1/grid, alpha=0.5, lw=1, label="inverse demand curve")  # P(x) = 1/x
ax.plot(grid, prices, 'k-', alpha=0.5, lw=2, label=r'$p^*$', color='blue')  # Equilibrium prices
ax.legend()
ax.set_xlabel('$x$', fontsize=12)
ax.set_ylabel('Price', fontsize=12)

plt.show()


In [ ]:
# Assuming the inverse demand function D(p) = 1 / p
def D(p):
    return 1 / p

# Define the price function p_star as before
p_star = interp1d(grid, prices, fill_value='extrapolate', bounds_error=False)

# Modify the carry_over function to use the inverse of p_star
def carry_over(x):
    return α * (x - D(p_star(x)))

# Assuming beta_dist is already defined earlier in your code
# and a, c are constants from your model
def generate_cp_ts(init=1, n=50):
    X = np.empty(n)
    X[0] = init
    for t in range(n-1):
            Z = a + c * beta_dist.rvs()
            X[t+1] = carry_over(X[t]) + Z
    return X  # Return the series of X values

# Plot the prices over time
fig, ax = plt.subplots()
ax.plot(generate_cp_ts(), label="price")
ax.set_xlabel("time")
ax.legend()
plt.show()


In [ ]:
# openbb library examples
def usbonds():
    bonds = openbb.economy.usbonds()
    bonds = bonds[::-1]
    bonds['Yld (%)'] = bonds['Yld (%)'].astype(float)
    return bonds

df = usbonds()

fig = go.Figure(data=go.Scatter(x=df[' '], y=df['Yld (%)']))
fig.update_layout(title="US Yield Curve", xaxis_title="", yaxis_title="Yield %")

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders


In [ ]:

# # Email configuration
# sender_email = 'mbquercioli@gmail.com'
# sender_password = 'Quarkmail424!!'
# recipient_email = 'fundmanager@tradeshare.io'
# subject = 'Image Attachment'

# # Create a MIME message
# message = MIMEMultipart()
# message['From'] = sender_email
# message['To'] = recipient_email
# message['Subject'] = subject

# # Attach the image file
# image_filename = '/Users/mark/OpenBB/newplot.png'  # Replace with your image file
# attachment = open(image_filename, 'rb').read()

# image_part = MIMEImage(attachment)
# image_part.add_header('Congoldt-Disposition', 'attachment', filename=image_filename)
# message.attach(image_part)

# # Send the email
# try:
#     smtp_server = smtplib.SMTP('smtp.gmail.com', 587)
#     smtp_server.starttls()
#     smtp_server.login(sender_email, sender_password)
#     smtp_server.sendmail(sender_email, recipient_email, message.as_string())
#     smtp_server.quit()
#     print('Email sent successfully')
# except Exception as e:
#     print(f'Error sending email: {str(e)}')


In [ ]:
gold = openbb.futures.historical(symbols='GC', start_date='2022-01-01', end_date=formatted_date)

gold['daily_rtn'] = gold['Adj Close'].pct_change()

gold['state'] = np.where(gold['daily_rtn'] > 0, "up", "down")
gold.tail()

In [ ]:
# gold futures 
# markhov chain logic 
# count logic from state 
# up-up, up-dwn, dwn-up, dwn-dwn
up_cnts = len(gold[gold['state'] == 'up'])
dwn_cnts = len(gold[gold['state'] == 'down'])
up_to_up = len(gold[(gold['state'] == 'up') & (gold['state'].shift(-1) == 'up')])/len(gold.query('state=="up"'))
dwn_to_up = len(gold[(gold['state'] == 'up') & (gold['state'].shift(-1) == 'down')])/len(gold.query('state=="up"'))

up_to_dwn = len(gold[(gold['state'] == 'down') & (gold['state'].shift(-1) == 'up')])/len(gold.query('state=="down"'))

dwn_to_dwn = len(gold[(gold['state'] == 'down') & (gold['state'].shift(-1) == 'down')])/len(gold.query('state=="down"'))


In [ ]:
# from col to index 
trans_matrix_gold = pd.DataFrame(
    {
        "up": [up_to_up, up_to_dwn],
        "down":[dwn_to_up, dwn_to_dwn]
    },
    index=["_up", "_down"]
)
trans_matrix_gold

In [ ]:
gold['Close'].plot()

In [ ]:
len(gold[(gold['state'] == 'down') & (gold['state'].shift(-1) == 'up') & (gold['state'].shift(-2) == 'up') ])/len(gold[(gold['state'].shift(1) =='up') & (gold['state'].shift(2) == 'up')])


In [ ]:
# simple mean reversion strategy
# Sample data setup (you probably have your own data loading method)
# gold = pd.read_csv('path_to_your_data.csv')

# Calculate the True Range
gold['High_Low'] = gold['High'] - gold['Low']
gold['High_PrevClose'] = abs(gold['High'] - gold['Close'].shift(1))
gold['Low_PrevClose'] = abs(gold['Low'] - gold['Close'].shift(1))
gold['TR'] = gold[['High_Low', 'High_PrevClose', 'Low_PrevClose']].max(axis=1)

# Calculate Average True Range (ATR) using a 14-period lookback by default
lookback = 14
gold['ATR'] = gold['TR'].rolling(window=lookback).mean()

# Ensure 'ATR' does not contain zeros to prevent dividing by zero
gold['ATR'] = gold['ATR'].replace(0, np.nan)

# Calculate the 'Dynamic_Lookback' column
gold['Dynamic_Lookback'] = lookback + (1 / gold['ATR'])

# Handle NaN values:

# 1. Drop rows with NaN values in 'Dynamic_Lookback'
gold.dropna(subset=['Dynamic_Lookback'], inplace=True)

# 2. OR, fill NaN in 'Dynamic_Lookback' with a default value (e.g., 20)
# gold['Dynamic_Lookback'].fillna(20, inplace=True)

# Convert the 'Dynamic_Lookback' column to integer
gold['Dynamic_Lookback'] = gold['Dynamic_Lookback'].astype(int)


In [ ]:
# simple mean reversion strategy
fast_window = 30
slow_window = 48
threshold = 0.0236  # 3% deviation from the mean, need to make some 

# Calculate 30-day moving average, fast and slow 
gold['FMA'] = gold['Close'].rolling(window=fast_window).mean()
gold['SMA'] = gold['Close'].rolling(window=slow_window).mean()

gold_df = gold[['Open','High', 'Low', 'Close']]
gold_bb = openbb.ta.bbands(gold_df, n_std=2.1).reset_index(drop=True)

# Calculate thresholds
# gold['Buy_Threshold'] = gold['FMA'] * (1 - threshold)
# gold['Sell_Threshold'] = gold['FMA'] * (1 + threshold)

gold['Buy_Threshold'] = gold['FMA'] - 0.681 * gold['ATR']
gold['Sell_Threshold'] = gold['FMA'] + 0.681 * gold['ATR']
gold.tail(5)


# Signal: Price breaks below Red SMA, Red Stoch breaks below 75, RSI breaks it’s trend line below 50

In [ ]:
# Create Buy/Sell signals
gold['Buy_Signal'] = np.where((gold['Close'] < (gold['Buy_Threshold'])) & (gold['SMA'] < gold['FMA']), 1, 0)
gold['Sell_Signal'] = np.where((gold['Close'] > (gold['Sell_Threshold'])) & (gold['SMA'] > gold['FMA']), 1, 0)

# gold['Entry'] = np.where((gold['Close'] < (gold['SMA'] - 2 * gold['ATR'])) & (gold['FMA'] > gold['SMA']), 1, 0)
# gold['Exit'] = np.where((gold['Close'] > (gold['SMA'] + 2 * gold['ATR'])) & (gold['FMA'] < gold['SMA']), 1, 0)
gold['Entry'] = np.where((gold['Close'] < (gold['SMA'] - 2 * gold['ATR'])), 1, 0)
gold['Exit'] = np.where((gold['Close'] > (gold['SMA'] + 2 * gold['ATR'])), 1, 0)

In [ ]:
startDate = '2023-03-30'
endDate = formatted_date
from datetime import datetime

today = datetime.today()

if 'Date' not in gold.columns:
    # Reset the index and name the new column 'Date'
    gold = gold.reset_index().rename(columns={'index': 'Date'})
    # Convert the 'Date' column to datetime
    gold['Date'] = pd.to_datetime(gold['Date'])


# # Filter the DataFrame
filtered_df = gold[(gold['Date'] >= startDate) & (gold['Date'] <= endDate)].reset_index(drop=True)
filtered_df.tail()

In [ ]:
tick_value = 10  # $10 per tick
ticks_per_point = 10  # 10 ticks make a point

initial_portfolio = 50000  # Starting with $100,000
cash = initial_portfolio
contracts_held = 2
portfolio_values = []
buy_dates = []
buy_dates_test = []
buy_prices = []
sell_dates = []
sell_dates_test = []
sell_prices = []

in_position = False

for index, row in filtered_df.iterrows():
    if row['Buy_Signal'] == 1:

        # print(f"Buying on {row['Date']} at price {row['Close']}")
        buy_dates.append(row['Date'])
        buy_prices.append(row['Close'])
        in_position = True
        
        if cash >= (row['Close'] * tick_value * ticks_per_point):
            contracts_to_buy = (cash // (row['Close'] * tick_value * ticks_per_point))
            contracts_held += contracts_to_buy
            cash -= contracts_to_buy * row['Close'] * tick_value * ticks_per_point
            
    if row['Entry'] == 1:

        # print(f"Entering on {row['Date']} at price {row['Close']}")
        buy_dates_test.append(row['Date'])
        buy_prices.append(row['Close'])
        in_position = True

    if row['Sell_Signal'] == 1:

        # print(f"Selling on {row['Date']} at price {row['Close']}")
        sell_dates.append(row['Date'])
        sell_prices.append(row['Close'])
        in_position = False

        if contracts_held > 0:
            cash += contracts_held * row['Close'] * tick_value * ticks_per_point
            contracts_held = 0

    if row['Exit'] == 1:

        # print(f"Exiting on {row['Date']} at price {row['Close']}")
        sell_dates_test.append(row['Date'])
        sell_prices.append(row['Close'])
        in_position = False

        if contracts_held > 0:
            cash += contracts_held * row['Close'] * tick_value * ticks_per_point
            contracts_held = 0
    portfolio_value = cash + contracts_held * row['Close'] * tick_value * ticks_per_point
    portfolio_values.append(portfolio_value)

    
# print("Porfolio Value: ", portfolio_value)

In [ ]:
fig = px.scatter(filtered_df, x=filtered_df['Date'], y='Close', title='MRS on GC')

# Add lines for ngosing prices and moving average
fig.add_scatter(x=filtered_df['Date'], y=filtered_df['Close'], mode='lines', name='Close')
fig.add_scatter(x=filtered_df['Date'], y=filtered_df['FMA'], mode='lines', name='30-day MA', line=dict(dash='dash'))
fig.add_scatter(x=filtered_df['Date'], y=filtered_df['SMA'], mode='lines', name='48-day MA', line=dict(dash='dot'))


# Add buy signals
fig.add_scatter(x=buy_dates, y=buy_prices, mode='markers', name='Buy Signal', marker=dict(size=20, symbol='triangle-up', color='green'))
fig.add_scatter(x=buy_dates_test, y=buy_prices, mode='markers', name='Entry', marker=dict(size=20, symbol='triangle-up', color='blue'))


# Add sell signals
fig.add_scatter(x=sell_dates, y=sell_prices, mode='markers', name='Sell Signal', marker=dict(size=20, symbol='triangle-down', color='red'))
fig.add_scatter(x=sell_dates_test, y=sell_prices, mode='markers', name='Exit', marker=dict(size=20, symbol='triangle-down', color='orange'))

fig.show()


In [ ]:
#profit and learn 

In [ ]:
import dash
import dash_ag_grid as dag
import dash_mantine_components as dmc
import pandas as pd
from dash import Input, Output, State, dcc, html, no_update
from dash_iconify import DashIconify

In [ ]:
future = openbb.futures.historical(symbols='NG', start_date='2010-01-01', end_date=formatted_date)
future

In [ ]:
future = future.reset_index().rename(columns={'index': 'Date'})
future['Date'] = pd.to_datetime(future['Date'])

In [ ]:
# ng_2312 = openbb.futures.historical(symbols = ['NG'], expiry = '2023-12', start_date='2022-01-01', end_date='2023-10-30')
# ng_2312 = ng_2312.rename(columns={'Adj Close':'2023-12'})
ng_2412 = openbb.futures.historical(symbols = ['NG'], expiry = '2024-12', start_date='2022-01-01', end_date=formatted_date)
ng_2412 = ng_2412.rename(columns={'Adj Close':'2024-12'})
ng_2512 = openbb.futures.historical(symbols = ['NG'], expiry = '2025-12', start_date='2022-01-01', end_date=formatted_date)
ng_2512 = ng_2512.rename(columns={'Adj Close':'2025-12'})
ng_2612 = openbb.futures.historical(symbols = ['NG'], expiry = '2026-12', start_date='2022-01-01', end_date=formatted_date)
ng_2612 = ng_2612.rename(columns={'Adj Close':'2026-12'})
ng_2712 = openbb.futures.historical(symbols = ['NG'], expiry = '2027-12', start_date='2022-01-01', end_date=formatted_date)
ng_2712 = ng_2712.rename(columns={'Adj Close':'2027-12'})
ng_2812 = openbb.futures.historical(symbols = ['NG'], expiry = '2028-12', start_date='2022-01-01', end_date=formatted_date)
ng_2812 = ng_2812.rename(columns={'Adj Close':'2028-12'})
ng_2912 = openbb.futures.historical(symbols = ['NG'], expiry = '2029-12', start_date='2022-01-01', end_date=formatted_date)
ng_2912 = ng_2912.rename(columns={'Adj Close':'2029-12'})
ng_3012 = openbb.futures.historical(symbols = ['NG'], expiry = '2030-12', start_date='2022-01-01', end_date=formatted_date)
ng_3012 = ng_3012.rename(columns={'Adj Close':'2030-12'})

historical = pd.DataFrame(data = [ng_2412['2024-12'],ng_2512['2025-12'],ng_2612['2026-12'],ng_2712['2027-12'],ng_2812['2028-12'],ng_2912['2029-12'],ng_3012['2030-12']]).transpose()
historical = historical.dropna()

historical

In [ ]:
# Reset the index (so 'date' becomes a column) and then melt the data
df_melted_all_future = historical.reset_index().melt(id_vars='Date', value_name='Price', var_name='Expiration')
fig = px.line(df_melted_all_future, x='Date', y='Price', color='Expiration', title='{} Futures Expiry'.format('NG'))
fig.show()

In [ ]:
# gc_2312 = openbb.futures.historical(symbols = ['GC'], expiry = '2023-12', start_date='2022-01-01', end_date='2023-10-30')
# gc_2312 = gc_2312.rename(columns={'Adj Close':'2023-12'})
gc_2412 = openbb.futures.historical(symbols = ['GC'], expiry = '2024-12', start_date='2022-01-01', end_date=formatted_date)
gc_2412 = gc_2412.rename(columns={'Adj Close':'2024-12'})
gc_2512 = openbb.futures.historical(symbols = ['GC'], expiry = '2025-12', start_date='2022-01-01', end_date=formatted_date)
gc_2512 = gc_2512.rename(columns={'Adj Close':'2025-12'})
gc_2612 = openbb.futures.historical(symbols = ['GC'], expiry = '2026-12', start_date='2022-01-01', end_date=formatted_date)
gc_2612 = gc_2612.rename(columns={'Adj Close':'2026-12'})
gc_2712 = openbb.futures.historical(symbols = ['GC'], expiry = '2027-12', start_date='2022-01-01', end_date=formatted_date)
gc_2712 = gc_2712.rename(columns={'Adj Close':'2027-12'})
gc_2812 = openbb.futures.historical(symbols = ['GC'], expiry = '2028-12', start_date='2022-01-01', end_date=formatted_date)
gc_2812 = gc_2812.rename(columns={'Adj Close':'2028-12'})
gc_2912 = openbb.futures.historical(symbols = ['GC'], expiry = '2029-12', start_date='2022-01-01', end_date=formatted_date)
gc_2912 = gc_2912.rename(columns={'Adj Close':'2029-12'})
gc_3012 = openbb.futures.historical(symbols = ['GC'], expiry = '2030-12', start_date='2022-01-01', end_date=formatted_date)
gc_3012 = gc_3012.rename(columns={'Adj Close':'2030-12'})

historical = pd.DataFrame(data = [gc_2412['2024-12'],gc_2512['2025-12'],gc_2612['2026-12']]).transpose()
historical = historical.dropna()

historical

In [ ]:
# Ensure the 'country' column has the full country name
# df_melted['country'] = df_melted['variable'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

# # Drop the now redundant 'variable' column
# df_melted.drop('variable', axis=1, inplace=True)

# # Now you can plot the data using Plotly Express

In [ ]:
# Reset the index (so 'date' becomes a column) and then melt the data
df_melted_all_future = historical.reset_index().melt(id_vars='Date', value_name='Price', var_name='Expiration')
fig = px.line(df_melted_all_future, x='Date', y='Price', color='Expiration', title='{} Futures Expiry'.format('GC'))
fig.show()


In [ ]:
weather = pd.read_csv('/Users/mark/OpenBB/Scripts/chartgpt/weather_daily.csv')
weather.head()

In [ ]:
df_all = pd.concat([weather, future],axis=1)

In [ ]:
df_continuous = df_all.select_dtypes(include=['float64'])

In [ ]:
corr = df_continuous.corr()
corr

In [ ]:
corr = df_continuous.corr()

fig = px.imshow(corr,
                x=corr.columns,
                y=corr.columns,
                text_auto=True,
                aspect="auto",
                color_continuous_scale='RdBu_r', # This color scale is diverging from red to blue
                )

# Update layout for a better view
fig.update_layout(title='Correlation Matrix Heatmap',
                  xaxis_title='Variables',    
                  yaxis=dict(
                    title='Variables',
                    title_standoff=15, # Adjust this value as needed to increase space
    ),
                  coloraxis_colorbar=dict(title='Correlation Coefficient',x=1.2)
                                          ,    
    width=1400, # Width of the figure in pixels
    height=1200, # Height of the figure in pixels
    autosize=False # Allow manual resizing)
)

fig.show()

In [ ]:
# df_melted_all_future.to_csv('ng_futures.csv', index=False)  # Set index=False to exclude the index column from the CSV


In [ ]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv('/Users/mark/OpenBB/Scripts/keys.env')
api_key = os.getenv('API_OPENAI_KEY')
acct_sid = os.getenv('ACCT_SID')
auth_token = os.getenv('AUTH_TOKEN')

In [ ]:
from twilio.rest import Client
number = "18557630954"
to = "5179306506"

ngient = Client(acct_sid, auth_token)

ngient
message = 'Hey, this is mark!'

message = ngient.messages.create(
    body=message,
    from_ = number,
    to = '3039280491'
)

print(f"Message sent with SID: {message.sid}")

In [ ]:
# temp data from meteo NYC
endpoint_ny = "https://archive-api.open-meteo.com/v1/archive?latitude=54.52&longitude=105.25&start_date=2023-10-26&end_date=2023-11-09&hourly=temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,snow_depth,et0_fao_evapotranspiration&daily=temperature_2m_max,temperature_2m_min,temperature_2m_mean,sunrise,sunset,precipitation_sum,snowfall_sum,shortwave_radiation_sum,et0_fao_evapotranspiration&temperature_unit=fahrenheit&timezone=auto"
response = requests.get(endpoint_ny)
# Check the response status code
if response.status_code == 200:
# The request was successful
    data = response.json()
    
else:
    # The request failed
    print(response.status_code)


df = pd.DataFrame(data['daily'])
print(df.columns)

In [ ]:
fig = px.line(df, x='time', y=['temperature_2m_max', 'temperature_2m_min',
       'temperature_2m_mean','precipitation_sum',
       'snowfall_sum', 'shortwave_radiation_sum',
       'et0_fao_evapotranspiration'])
fig.show()

In [ ]:
import plotly.express as px
weather.head()

In [ ]:
# Plot histograms for all numerical data
# for column in weather.select_dtypes(include=['float64', 'int64']).columns:
#     fig = px.histogram(weather, x=column, title=f'Histogram of {column}')
#     fig.show()

In [ ]:
# Kp and Ap data from api
url = 'https://kp.gfz-potsdam.de/app/json/'
params1 = {
    'start': '2010-01-30T00:00:00Z',
    'end': '2022-02-02T23:59:59Z',
    'index': 'Kp',  # Add your indices separated by commas or as required by the API
    'status': 'def'
}
params2 = {
    'start': '2010-01-30T00:00:00Z',
    'end': '2022-02-02T23:59:59Z',
    'index': 'Ap',  # Add your indices separated by commas or as required by the API
    'status': 'def'
}
kp = requests.get(url, params=params1)
ap = requests.get(url, params=params2)
if kp.status_code == 200 and ap.status_code == 200:
# The request was successful
    print('Request Sucessful: 200')
    kp_data = kp.json()
    ap_data = ap.json()
    
else:
    # The request failed
    print(kp.status_code, ap.status_code)

In [ ]:
# Convert responses to DataFrames
df_kp = pd.DataFrame({
    'datetime': kp_data['datetime'],
    'Kp': kp_data['Kp']
})
df_ap = pd.DataFrame({
    'datetime': ap_data['datetime'],
    'Ap': ap_data['Ap']
})

# Convert 'datetime' to datetime objects and normalize (remove time)
df_kp['datetime'] = pd.to_datetime(df_kp['datetime']).dt.normalize()
df_ap['datetime'] = pd.to_datetime(df_ap['datetime']).dt.normalize()

# Join the DataFrames on 'datetime'
df_joined = pd.merge(df_kp, df_ap, on='datetime', how='inner')

# Set 'datetime' as index
df_final = df_joined.set_index('datetime')

df_daily = df_final.resample('D').mean()  # 'D' stands for day

df_daily.index = df_daily.index.normalize().strftime('%Y-%m-%d')
df_daily.reset_index(inplace=True)

df_daily

In [ ]:
df_all = pd.concat([future, df_daily],axis=1)
# df_all.head(30)

In [ ]:
fig = px.line(df_all, x='Date', y=['Adj Close', 'Kp','Ap'])
fig.show()

In [ ]:
df_cont = df_all.select_dtypes(include=['float64'])

In [ ]:
corr = df_cont.corr()

In [ ]:
fig = px.imshow(corr,
                x=corr.columns,
                y=corr.columns,
                text_auto=True,
                aspect="auto",
                color_continuous_scale='RdBu_r', # This color scale is diverging from red to blue
                )

# Update layout for a better view
fig.update_layout(title='Correlation Matrix Heatmap',
                  xaxis_title='Variables',    
                  yaxis=dict(
                    title='Variables',
                    title_standoff=15, # Adjust this value as needed to increase space
    ),
                  coloraxis_colorbar=dict(title='Correlation Coefficient',x=1.2)
                                          ,    
    width=1400, # Width of the figure in pixels
    height=1200, # Height of the figure in pixels
    autosize=False # Allow manual resizing)
)

# fig.show()

In [ ]:
df_cont[['Kp', 'Ap']].hist()


In [ ]:
def usbonds():
    bonds = openbb.economy.usbonds()
    bonds = bonds[::-1]
    bonds['Yld (%)'] = bonds['Yld (%)'].astype(float)
    return bonds

usbonds()

In [ ]:
bonds = openbb.economy.usbonds()
bonds

In [ ]:
import plotly.express as px
import pandas as pd
df = pd.read_csv('weather_hrly.csv')
# Filter the dataframe to include only the data from the last year
last_year_df = df[df['time'] >= '2022-12-21']

# Calculate the total amount of rain in the last year
total_rain = last_year_df['rain (inch)'].sum()

# Create a bar chart to visualize the total amount of rain in the last year
fig = px.bar(last_year_df, x='time', y='rain (inch)', title='Rainfall in the Last Year')
fig

# Print the total amount of rain in the last year
print(f"The total amount of rain in the last year is {total_rain} inches.")


In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('weather_daily.csv')
df.columns
# Convert 'time' to a datetime format if it's not already
df['time'] = pd.to_datetime(df['time'])

# Melt the DataFrame to a long format
df_long = df.melt(id_vars=['time'], 
                  value_vars=['temperature_2m_max (°F)', 'temperature_2m_min (°F)', 'temperature_2m_mean (°F)', 'shortwave_radiation_sum (MJ/m²)'],
                  var_name='Measurement', value_name='Value')

# Create the scatter plot
fig = px.scatter(df_long, x='time', y='Value', color='Measurement', title='Temperature and Solar Radiation')

# Update layout
fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Temperature (°F) / Solar Radiation (MJ/m²)',
    legend_title='Variables'
)

fig.show()


In [ ]:
import plotly.express as px

df = pd.read_csv('ng_futures.csv')
# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])
df.dtypes
# # Extract the month and year from the 'Date' column
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Calculate the average price monthly
monthly_avg_price = df.groupby(['Year', 'Month'])['Price'].mean().reset_index()

# Create the chart
fig = px.line(monthly_avg_price, x='Month', y='Price', color='Year', title='Average Price Monthly')
fig

In [ ]:
import plotly.express as px
# plot all price data by year 
# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Extract the year from the 'Date' column
df['Year'] = df['Date'].dt.year

# Create the plot
fig = px.line(df, x='Date', y='Price', color='Year')

# Show the plot
fig